In [68]:
import numpy as np
import seaborn as sns
from sklearn import datasets
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import AdaBoostClassifier

# Load the dataset
titanic = sns.load_dataset('titanic')

In [69]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [70]:
# Handling missing values: fill missing 'age' values with median
titanic['age'].fillna(titanic['age'].median(), inplace=True)

# Removing duplicates: check for duplicates and drop if any
titanic.drop_duplicates(inplace=True)

# Correcting data entry errors: replace any negative 'fare' values with the median fare
titanic.loc[titanic['fare'] < 0, 'fare'] = titanic['fare'].median()

# Converting data types: convert 'sex' variable to a categorical data type
titanic['sex'] = titanic['sex'].astype('category')

# Addressing outliers: remove rows where 'fare' is greater than 3 standard deviations from the mean
fare_mean = titanic['fare'].mean()
fare_std = titanic['fare'].std()
titanic = titanic[(titanic['fare'] >= fare_mean - 3 * fare_std) & (titanic['fare'] <= fare_mean + 3 * fare_std)]

In [71]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [76]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Index: 761 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     761 non-null    int64   
 1   pclass       761 non-null    int64   
 2   sex          761 non-null    category
 3   age          761 non-null    float64 
 4   sibsp        761 non-null    int64   
 5   parch        761 non-null    int64   
 6   fare         761 non-null    float64 
 7   embarked     759 non-null    object  
 8   class        761 non-null    category
 9   who          761 non-null    object  
 10  adult_male   761 non-null    bool    
 11  deck         185 non-null    category
 12  embark_town  759 non-null    object  
 13  alive        761 non-null    object  
 14  alone        761 non-null    bool    
dtypes: bool(2), category(3), float64(2), int64(4), object(4)
memory usage: 69.7+ KB


In [72]:
titanic.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

In [77]:
cat_columns = ['pclass', 'sex', 'sibsp', 'parch', 'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town', 'alive', 'alone']
num_columns = ['age', 'fare']

target = 'survived'

# In Class Code

In [74]:
X_train, X_test, y_train, y_test = train_test_split(titanic[cat_columns + num_columns], titanic[target], test_size=0.2, random_state=42)

In [83]:
cat_transformer = Pipeline(steps = [('onehot', OneHotEncoder(handle_unknown='ignore'))])
num_transformer = Pipeline(steps = [('scaler', StandardScaler())])
preprocessor = ColumnTransformer(transformers = [('cat',cat_transformer, cat_columns),
                                                 ('num', num_transformer, num_columns)])

In [84]:
d_tree = DecisionTreeClassifier(random_state=42, max_depth=8)
d_tree.fit(X_train, y_train)
y_pred = d_tree.predict(X_test)

ValueError: Cannot cast object dtype to float32

In [79]:
# Number of bootstrap samples and trees in the ensemble
n_samples = len(X_train)
n_trees = 50

def bootstrap_sample(X, y):
    indices = np.random.choice(len(X), size=len(X), replace=True)
    return X[indices], y[indices]

In [81]:
# Train multiple decision trees on different bootstrap samples
trees = []
for _ in range(n_trees):
    X_sample, y_sample = bootstrap_sample(X_train, y_train)
    
    # Clone the pipeline for each tree
    #cloned_pipeline = Pipeline(steps=[('scaler', StandardScaler()), ('clf', DecisionTreeClassifier())])
    preprocessor.fit(X_sample, y_sample)
    trees.append(preprocessor)

KeyError: "None of [Index([121, 567, 126, 506, 429,  94, 573, 563, 297, 335,\n       ...\n       280, 382, 457, 301, 535, 420, 362, 274, 393, 542],\n      dtype='int32', length=608)] are in the [columns]"